# 0- Setup

In [1]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]
        
        
%load_ext autoreload
%autoreload 2

#magic commands 

- Naming convention:
  - g: german dataset 
  - int: Initial dataset without anomalies removal
  - GHF: dataset containing onservables 
  - grid: grid containing observables 

# 1- Import libraries

In [2]:
#Logging and Warnings
import warnings  # Provides a way to handle warning messages that may occur during the execution of the code.
import logging  # Module for flexible logging of messages and errors.

#Visualization
import matplotlib.pyplot as plt  # Plotting library for creating various types of plots and visualizations.

#File and Path Operations
from pathlib import Path  # Object-oriented filesystem paths.
import joblib  # Module for saving and loading Python objects.
import pickle  # Module for serializing and deserializing Python objects.


#Data Manipulation and Analysis
import pandas as pd  # Data manipulation and analysis library.
import numpy as np  # Fundamental package for scientific computing with Python.
import xarray as xr # For labeled multi-dimensional arrays and datasets.

#Machine Learning
from sklearn import (
    compose,  
    ensemble,  
    preprocessing, 
    model_selection,
    metrics
)


from QRF.wrapper.qrf_wrapper import QuantileRegressionForest


#Logging Configuration and Ignoring Warnings
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Ignore warnings
warnings.filterwarnings('ignore')


#Directory and Styling
DIR = Path().resolve()  # Current working directory.


# 2- Variables and constants

### Constants relative to figures

In [3]:

# Set global plt parameters
plt_params = {
    'figure.titlesize': 18,
    'axes.titlesize': 16,
    'axes.labelsize': 16,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.title_fontsize': 13,
    'legend.fontsize': 12,
    'axes.linewidth': 1.2,
    'axes.edgecolor': 'black',
    'axes.titleweight': 'bold',  
    'axes.labelweight': 'bold',  
    'font.weight': 'bold',
    'font.size': 12,
    'grid.color': 'gray',
    'grid.alpha': 0.7,
    #'tick.direction': 'in',
    #'figure.figsize': (8, 6),
    'savefig.dpi': 300,
}


plt.rcParams.update(plt_params)

# Sub figures
sub_figs = list('abcdefghijklmnopqrstuvwxyz123456789')



### Constants relative to datasets

In [4]:
#Initialize a dictionary called obs with different lists as values.
#obs are organized alphabetically
obs_dict = {
'REF_CODE_UNTRANS': {'BG': 'BG',  'CTD_vb35': 'CTD_vb35',  'DEM': 'DEM',  'MAG_CLASS': 'MAG',  'FA': 'FA',  'GEOID_A': 'GEOID_A',  'LAB': 'LAB',  'MOHO': 'MOHO',  'RHO_L': 'RHO_L',  'SEDIMENT': 'SEDIMENT',  'SI': 'SI',  'VOLCS_DIST_W': 'VOLCS_DIST',  'VP': 'VP_VELOCITY',  'VS': 'VS_VELOCITY',  'STRESS_DIST_W': 'STRESS_DIST'},
'LABELS': {'BG': 'Bouguer Anomaly',  'CTD_vb35': 'Curie Depth',  'DEM': 'DEM',  'MAG_CLASS': 'Magnetic Anomaly',  'FA': 'Free Air Anomaly',  'GEOID_A': 'Geoid',  'LAB': 'LAB Depth',  'MOHO': 'Moho Depth',  'RHO_L': 'Litho. ρ',  'SEDIMENT': 'Sediment Thickness',  'SI': 'Shape Index',  'VOLCS_DIST_W': 'Proximity to Volcano',  'VP': 'Δ$P_v$ @150Km tomography',  'VS': 'Δ$S_v$ @150km tomography',  'STRESS_DIST_W': 'Proximity to Faults'},
'LABELS_gmt': {'BG': 'Bouguer\tAnomaly',  'CTD_vb35': 'Curie\tDepth',  'DEM': 'DEM',  'MAG_CLASS': 'Magnetic\tAnomaly',  'FA': 'Free\tair\tanomaly',  'GEOID_A': 'Geoid',  'LAB': 'LAB\tDepth',  'MOHO': 'Moho\tDepth',  'RHO_L': 'Lith.\t@~r@ ',  'SEDIMENT': 'Sediment\tThickness',  'SI': 'Shape\tindex',  'VOLCS_DIST_W': 'Proximity\tto\tvolcano',  'VP': '@~D@~P@_v@\t@@150km\ttomography',  'VS': '@~D@~S@_v@\t@@150km\ttomography',  'STRESS_DIST_W': 'Proximity\tto\tfaults'},
'LABELS_gmt_UNTRANS': {'BG': 'Bouguer\tAnomaly',  'CTD_vb35': 'Curie\tDepth',  'DEM': 'DEM',  'MAG_CLASS': 'Magnetic\tAnomaly',  'FA': 'Free\tair\tAnomaly',  'GEOID_A': 'Geoid',  'LAB': 'LAB\tDepth',  'MOHO': 'Moho\tDepth',  'RHO_L': 'Lith.\t@~r@ ',  'SEDIMENT': 'Sediment\tThickness',  'SI': 'Shape\tindex',  'VOLCS_DIST_W': 'Distance\tto\tVolcano',  'VP': 'P@_v@\t@@150km\ttomography',  'VS': 'S@_v@\t@@150km\ttomography',  'STRESS_DIST_W': 'Distance\tto\tfaults'},
'UNITS': {'BG': 'mGal',  'CTD_vb35': 'km',  'DEM': 'm',  'MAG_CLASS': 'f(nT)',  'FA': 'mGal',  'GEOID_A': 'm',  'LAB': 'km',  'MOHO': 'km',  'RHO_L': 'kgm$^{-3}$',  'SEDIMENT': 'km',  'SI': 'si',  'VOLCS_DIST_W': 'f(Km)',  'VP': '%',  'VS': '%',  'STRESS_DIST_W': 'f(km)'},
'UNITS_gmt': {'BG': 'mGal',  'CTD_vb35': 'km',  'DEM': 'm',  'MAG_CLASS': 'f(nT)',  'FA': 'mGal',  'GEOID_A': 'm',  'LAB': 'km',  'MOHO': 'km',  'RHO_L': 'kgm@+-3@',  'SEDIMENT': 'km',  'SI': 'si',  'VOLCS_DIST_W': 'f(Km)',  'VP': '%',  'VS': '%',  'STRESS_DIST_W': 'f(km)'},
'UNITS_gmt_UNTRANS': {'BG': 'mGal',  'CTD_vb35': 'km',  'DEM': 'm',  'MAG_CLASS': 'nT',  'FA': 'mGal',  'GEOID_A': 'm',  'LAB': 'km',  'MOHO': 'km',  'RHO_L': 'kgm@+-3@',  'SEDIMENT': 'km',  'SI': 'si',  'VOLCS_DIST_W': 'Km',  'VP': 'ms@+-2@',  'VS': 'ms@+-2@',  'STRESS_DIST_W': 'km'},
'V_RANGE_W': {'BG': (-100, 100),  'CTD_vb35': (10, 40),  'DEM': (-2200, 2200),  'MAG_CLASS': (-0.4, 0.4),  'FA': (-100, 100),  'GEOID_A': (-30, 30),  'LAB': (0, 300),  'MOHO': (15, 60),  'RHO_L': (3.26, 3.36),  'SEDIMENT': (0, 10),  'SI': (-1, 1),  'VOLCS_DIST_W': (0, 1),  'VP': (-0.02, 0.02),  'VS': (-0.1, 0.1),  'STRESS_DIST_W': (0, 1)}, 'V_RANGE_G': {'BG': (-100, 100),  'CTD_vb35': (10, 40),  'DEM': (-2200, 2200),  'MAG_CLASS': (-0.4, 0.4),  'FA': (-100, 100),  'GEOID_A': (-30, 30),  'LAB': (50, 250),  'MOHO': (20, 50),  'RHO_L': (3.26, 3.36),  'SEDIMENT': (0, 10),  'SI': (-1, 1),
'VOLCS_DIST_W': (0, 1),  'VP': (8.1, 8.4),  'VS': (4.2, 4.7),  'STRESS_DIST_W': (0, 1)},
'V_RANGE_G_UNTRANS': {'BG': (-100, 100),  'CTD_vb35': (10, 40),  'DEM': (-2200, 2200),  'MAG_CLASS': (-200, 200),  'FA': (-100, 100),  'GEOID_A': (-30, 30),  'LAB': (50, 250),  'MOHO': (20, 50),  'RHO_L': (3.26, 3.36),  'SEDIMENT': (0, 10),  'SI': (-1, 1),  'VOLCS_DIST_W': (0, 300),  'VP': (8.1, 8.4),  'VS': (4.2, 4.7),  'STRESS_DIST_W': (0, 300)},
'CMAPS_gmt': {'BG': 'SCM/vik',  'CTD_vb35': 'SCM/bamako',  'DEM': 'gmt/geo',  'MAG_CLASS': 'SCM/bilbao',  'FA': 'SCM/vik',  'GEOID_A': 'SCM/vik',  'LAB': 'SCM/bamako',  'MOHO': 'SCM/bamako',  'RHO_L': 'SCM/batlow',  'SEDIMENT': 'SCM/davos',  'SI': 'SCM/broc',  'VOLCS_DIST_W': 'SCM/broc',  'VP': 'SCM/roma',  'VS': 'SCM/roma',  'STRESS_DIST_W': 'SCM/broc'},
'LABELS_REDUCED': {'BG': 'Bouguer Anom.',  'CTD_vb35': 'Curie Depth',  'DEM': 'Topography',  'MAG_CLASS': 'Magnetic Anom.',  'FA': 'Free Air Anom.',  'GEOID_A': 'Geoid',
'LAB': 'LAB Depth',  'MOHO': 'Moho Depth',  'RHO_L': 'Litho. ρ',  'SEDIMENT': 'Sediment Thick.',  'SI': 'Shape Index',  'VOLCS_DIST_W': 'Prox. to Volcanos',  'VP': 'P wave tomogr.',  'VS': 'S wave tomogr.',  'STRESS_DIST_W': 'Prox. to Faults'}
}


# Set OBS_REF as index, making it easy to access by referencing observable
obs_df = pd.DataFrame(obs_dict)

# Set OBS_REF as index, making it easy to access by referencing observable
obs_df.sort_values(by='UNITS_gmt', ascending=True)

,REF_CODE_UNTRANS,LABELS,LABELS_gmt,LABELS_gmt_UNTRANS,UNITS,UNITS_gmt,UNITS_gmt_UNTRANS,V_RANGE_W,V_RANGE_G,V_RANGE_G_UNTRANS,CMAPS_gmt,LABELS_REDUCED
VP,VP_VELOCITY,Δ$P_v$ @150Km tomography,@~D@~P@_v@\t@@150km\ttomography,P@_v@\t@@150km\ttomography,%,%,ms@+-2@,"(-0.02, 0.02)","(8.1, 8.4)","(8.1, 8.4)",SCM/roma,P wave tomogr.
VS,VS_VELOCITY,Δ$S_v$ @150km tomography,@~D@~S@_v@\t@@150km\ttomography,S@_v@\t@@150km\ttomography,%,%,ms@+-2@,"(-0.1, 0.1)","(4.2, 4.7)","(4.2, 4.7)",SCM/roma,S wave tomogr.
VOLCS_DIST_W,VOLCS_DIST,Proximity to Volcano,Proximity\tto\tvolcano,Distance\tto\tVolcano,f(Km),f(Km),Km,"(0, 1)","(0, 1)","(0, 300)",SCM/broc,Prox. to Volcanos
STRESS_DIST_W,STRESS_DIST,Proximity to Faults,Proximity\tto\tfaults,Distance\tto\tfaults,f(km),f(km),km,"(0, 1)","(0, 1)","(0, 300)",SCM/broc,Prox. to Faults
MAG_CLASS,MAG,Magnetic Anomaly,Magnetic\tAnomaly,Magnetic\tAnomaly,f(nT),f(nT),nT,"(-0.4, 0.4)","(-0.4, 0.4)","(-200, 200)",SCM/bilbao,Magnetic Anom.
RHO_L,RHO_L,Litho. ρ,Lith.\t@~r@,Lith.\t@~r@,kgm$^{-3}$,kgm@+-3@,kgm@+-3@,"(3.26, 3.36)","(3.26, 3.36)","(3.26, 3.36)",SCM/batlow,Litho. ρ
CTD_vb35,CTD_vb35,Curie Depth,Curie\tDepth,Curie\tDepth,km,km,km,"(10, 40)","(10, 40)","(10, 40)",SCM/bamako,Curie Depth
LAB,LAB,LAB Depth,LAB\tDepth,LAB\tDepth,km,km,km,"(0, 300)","(50, 250)","(50, 250)",SCM/bamako,LAB Depth
MOHO,MOHO,Moho Depth,Moho\tDepth,Moho\tDepth,km,km,km,"(15, 60)","(20, 50)","(20, 50)",SCM/bamako,Moho Depth
SEDIMENT,SEDIMENT,Sediment Thickness,Sediment\tThickness,Sediment\tThickness,km,km,km,"(0, 10)","(0, 10)","(0, 10)",SCM/davos,Sediment Thick.


In [5]:
to_drop =['GEOID_A', 'LAB', 'SEDIMENT', 'SI', 'VP']
obs_df = obs_df.drop(labels=to_drop)

In [6]:
# Read CSV file and convert columns to appropriate data types
ghf_f = DIR / 'Dataset' / 'Preprocessed' / f'ghf_6_int.csv' 
ghf_df = pd.read_csv(ghf_f, sep='\t')

# Set variable values
TARGET       = 'GHF'
TARGET_LABEL = 'Heat Flow'
COORDs       = ['lon', 'lat']
GRID_INDEX_G = 'grid_index'
GHF_BOUNDS   = (int(ghf_df[TARGET].min()), int(np.ceil(ghf_df[TARGET].max())))
                

# Create full header names with short labels of observables for Germany
observables_lst   = obs_df.index.tolist()
observables_g_lst = observables_lst + COORDs + [GRID_INDEX_G, TARGET]

# Create header names with short labels of observables
observables_ghf_lst = observables_lst + [TARGET]

# Create full header names with long labels of observables for World and germnay
labels_lst   = obs_df.loc[observables_lst, 'LABELS_REDUCED'].values.tolist()
labels_g_lst = labels_lst + ['lon', 'lat', GRID_INDEX_G, TARGET]

# Create header names with long labels of observables
labels_ghf_lst = labels_lst + [TARGET_LABEL]



### Load datasets

In [7]:
RANDOM_STATE = 42

#load german grid with all its observables
grid_g_xr = xr.load_dataset(DIR/'Grids'/"grid_g.nc")

#get observables from grid for prediction
grid_g_df = grid_g_xr.to_dataframe()


#get observables from GHF training
X = ghf_df[observables_lst]
y = ghf_df[TARGET]
weights = ghf_df['normalized_weights']


RANDOM_STATE = 42

# Set file paths
tuned_hyperparameters_f = DIR / 'Hyperparameters' / 'hyperparameters.pkl'


config_bl = { 'n_estimators' : 1000, "bootstrap": True, "criterion": "squared_error", "random_state": RANDOM_STATE,}
classes  = {
    "class_min" : GHF_BOUNDS[0], 
    "class_max" : GHF_BOUNDS[1], 
}


n_classes = GHF_BOUNDS[1] - GHF_BOUNDS[0]+1

# Load the best hyperparameters using pickle
with open(tuned_hyperparameters_f, 'rb') as f:
    tuned_hyperparameters = pickle.load(f)
    


# 2-  Modeling

pipeline.get_params then get what is the

regressor is the instance in general

regressor_ is the fitted instance

In [8]:
miss_rate =  7.79

excess, defiit = miss_rate - 1.89, miss_rate + 6.6


low_entropy = 0.71
hi_entropy = 0.77

grid_g_nn = (len(grid_g_xr.Y), len(grid_g_xr.X))

qrf = QuantileRegressionForest(**classes).set_params(**config_bl)
qrf.set_params(**tuned_hyperparameters)

#qrf.fit(X, y)
qrf.fit(X, y, sample_weight=weights)


qrf.calculate_metrics_and_uncertainties(
             grid_g_df[observables_lst].values, grid_g_df, grid_g_df.index) 

# predict the HF value for each grid point in A_grid
grid_g_xr[f"AVG"]  = grid_g_df['AVG'].to_xarray()
grid_g_xr[f"MD"]   = grid_g_df['MD'].to_xarray()
grid_g_xr[f"LQ"]   = grid_g_df['LQ'].to_xarray()
grid_g_xr[f"UQ"]   = grid_g_df['UQ'].to_xarray()
grid_g_xr[f"UT"]   = grid_g_df['UT'].to_xarray()
grid_g_xr[f"IQSR"] = grid_g_df[f"IQSR"].to_xarray()

grid_g_xr[f"UT_n"]  = grid_g_xr[f"UT"]  / np.log(n_classes)
grid_g_xr['LU']     = (grid_g_xr['UT_n'] <= low_entropy).astype(int) & (grid_g_xr['IQSR'] <= excess).astype(int)
grid_g_xr['Error']  = (grid_g_xr['UT_n'] <= low_entropy).astype(int) & (grid_g_xr['IQSR'] > defiit).astype(int)
grid_g_xr['U']     = (grid_g_xr['UT_n'] > hi_entropy).astype(int)   & (grid_g_xr['IQSR'] <= excess).astype(int)
grid_g_xr['HU']     = (grid_g_xr['UT_n'] > hi_entropy).astype(int)   & (grid_g_xr['IQSR'] > defiit).astype(int)

           
    
logging.info(f'Terminated')



Estimating for samples: 100%|████████████████████████████████████████████████████| 8181/8181 [00:01<00:00, 4629.18it/s]
2024-08-14 23:44:23,604 - INFO - Calculating Quantiles has been terminated
2024-08-14 23:44:23,659 - INFO - Calculating uncertainties has been terminated - Elapsed time 0.06 s
2024-08-14 23:44:23,987 - INFO - Terminated


In [9]:
grid_g_nn = (len(grid_g_xr.Y), len(grid_g_xr.X))

qrf = QuantileRegressionForest(**classes).set_params(**config_bl)
qrf.set_params(**tuned_hyperparameters)

qrf.fit(X, y)

qrf.calculate_metrics_and_uncertainties(
             grid_g_df[observables_lst].values, grid_g_df, grid_g_df.index) 

# predict the HF value for each grid point in A_grid
grid_g_xr[f"MD_equal"]  = grid_g_df['MD'].to_xarray()
           
    
logging.info(f'Terminated')



Estimating for samples: 100%|████████████████████████████████████████████████████| 8181/8181 [00:01<00:00, 4651.03it/s]
2024-08-14 23:44:33,743 - INFO - Calculating Quantiles has been terminated - Elapsed time 10.08 s
2024-08-14 23:44:33,784 - INFO - Calculating uncertainties has been terminated - Elapsed time 0.04 s
2024-08-14 23:44:33,827 - INFO - Terminated


In [11]:
# Load the model for reproducibility
with open(model_path, 'rb') as f:
    qrf_model = pickle.load(f)